## 讯飞：广告图片素材分类算法挑战赛解题思路与数据EDA



### 一、赛题背景



讯飞AI营销是科大讯飞集团在数字广告领域发展的重要业务，基于深耕多年的人工智能技术和大数据积累，赋予营销智慧创新的大脑，以健全的产品矩阵和全方位的服务，帮助广告主用AI技术实现营销效能的全面提升，打造数字营销新生态。
目前，AI营销平台除自有媒体外，已在功能社交、休闲娱乐、专业进阶、衣食住行等类型的TOP媒体实现规模化覆盖，并且覆盖媒体数量仍在快速增长。如何确保在成百上千的媒体上投放符合要求的广告素材（例如，教育类APP需要投放适合青少年的广告内容），是素材审核迫切需要解决的问题。而作为最常见的广告素材类型之一，图片的自动分类将会极大提高审核效率。

### 二、赛事任务
本次分类算法任务中，讯飞AI营销将提供海量现网流量中的广告图片素材作为训练数据，参赛选手基于提供的训练数据构建模型，实现自动化广告图片素材分类。


### 三、数据说明
本次比赛为参赛选手提供的数据分为训练集、测试集、提交样例三类文件：

训练集：包含10万+广告素材图片，100多个类别，几十种图片尺寸；且图片已按类别放在不同文件夹内，文件夹名称即为素材图片的category_id。

测试集：包含1万张广告素材图片，放在同一个文件夹内，图片文件的名称即为image_id。

提交样例：表头为image_id和category_id的CSV文件，选手提交数据时需要将测试集的图片id与模型预测的类别id按样例格式填入CSV中，进行提交。

## 数据EDA


探索性数据分析（Exploratory Data Analysis，简称EDA），是指对已有的数据（原始数据）进行分析探索，通过作图、制表、方程拟合、计算特征量等手段探索数据的结构和规律的一种数据分析方法。一般来说，我们最初接触到数据的时候往往是毫无头绪的，不知道如何下手，这时候探索性数据分析就非常有效。


当我们拿到数据的时候，首先应该观察数据的分布情况，一般来说，图像分类任务的赛题通常是在类别不平衡上做文章。那么我们来看看这个数据集的数据分布情况。


In [ ]:
# 数据集解压
!cd 'data/data98270' && unzip -q train.zip

!cd 'data/data98441' && unzip -q test.zip

In [ ]:
## 数据EDA

import os
path = 'data/data98270/train'  #父文件夹路径
all_folds = os.listdir(path)   #解析出父文件夹中所有的文件名称，并以列表的格式输出，
#例如['add','common-mobile-web-app-icons.zip', '新建 Microsoft Word 文档.docx', 'arrow_down']
l = []
for i in range(len(all_folds)):
    fold_path = os.path.join(path,all_folds[i])  #将父文件夹路径加上子文件的名称，例如：'D:/testin/common-mobile-web-app-icons/add'
    if os.path.isdir(fold_path):   #判断该文件是否为文件夹
        count_fold = len(os.listdir(fold_path))
        #print(all_folds[i],count_fold)
        l.append((all_folds[i],count_fold))  #得到列表，列表里面是数组，数组里面是文件名称和该文件名称文件夹中图片个数
#print(l)
dic_file = dict(l)  #数组转成字典
#dic_file
txt_file = os.getcwd()+'\count.txt'  #os.getcwd()得到当前路径，并在当前路径下建一个txt文本文件
out = open(txt_file,'w')  #打开文本文件
for i in  dic_file:  #循环字典的键
    # out.write(i)  #写入键，既文件夹名称
    # out.write(': ')
    out.write(str(dic_file[i]))  #写入值，既文件夹名称下的图片个数
    out.write('\n')  #换行
out.close()  #关闭txt文本文件


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_table('count.txt',header=None,index_col=False)

# print(df[0])
data = df[0].sort_values()

plt.bar(range(len(data)), data)

plt.savefig("eda.jpg")
# plt.show()

![](https://ai-studio-static-online.cdn.bcebos.com/e0c07a51194642bc89eeedc35de821af759433122e4b444685a6b70f5cd92a6b)


* **解题思路**


**label shuffling**

&emsp;&emsp;首先对原始的图像列表，按照标签顺序进行排序；
然后计算每个类别的样本数量，并得到样本最多的那个类别的样本数。
根据这个最多的样本数，对每类都产生一个随机排列的列表；
然后用每个类别的列表中的数对各自类别的样本数求余，得到一个索引值，从该类的图像中提取图像，生成该类的图像随机列表；
然后把所有类别的随机列表连在一起，做个Random Shuffling，得到最后的图像列表，用这个列表进行训练。


![](https://ai-studio-static-online.cdn.bcebos.com/36dc85bab2c84602a04dec6fe7db3914a96c66546a7b4bb68f61b8eb77387c35)

**数据预处理**


此部分对数据进行预处理，生成我们需要的CSV文件。另外实现label shuffling策略，此处我们选择ResNet_vd版本，

In [ ]:
import os
import json
import cv2

rootDir = 'data/data98270/train'

list = []

image_id = []
category_id = []

def Test1(rootDir):
    list_dirs = os.walk(rootDir)
   

    img_id = 0
    for root, dirs, files in list_dirs:
        for d in dirs:
            # print(os.path.join(root, d))
            path = os.path.join(root, d)
            cat_id = int(d)
            for im in os.listdir(path):
                dict = {}
                img = cv2.imread(os.path.join(path, im))
                # print(os.path.join(path, im))
                img_h = img.shape[0]
                img_w = img.shape[1]
                dict['image_id'] = img_id
                img_id += 1
                dict['fpath'] = os.path.join(path, im)
                dict['im_height'] = img_h
                dict['im_width'] = img_w
                dict['category_id'] = cat_id
                list.append(dict)
                image_id.append(d+'/'+im)
                category_id.append(cat_id)

Test1(rootDir)

import pandas as pd

img = pd.DataFrame(image_id)
img = img.rename(columns = {0:"image_id"})
img['category_id'] = category_id

img.to_csv('train.csv', index=False)


In [ ]:
# 导入所需要的库
from sklearn.utils import shuffle
import os
import pandas as pd
import numpy as np
from PIL import Image

import paddle
import paddle.nn as nn
from paddle.io import Dataset
import paddle.vision.transforms as T
import paddle.nn.functional as F
from paddle.metric import Accuracy

import warnings
warnings.filterwarnings("ignore")

# 读取数据
train_images = pd.read_csv('train.csv')

# labelshuffling

def labelShuffling(dataFrame, groupByName = 'category_id'):

    groupDataFrame = dataFrame.groupby(by=[groupByName])
    labels = groupDataFrame.size()
    print("length of label is ", len(labels))
    maxNum = max(labels)
    lst = pd.DataFrame()
    for i in range(len(labels)):
        print("Processing label  :", i)
        tmpGroupBy = groupDataFrame.get_group(i)
        print(labels[2])

        createdShuffleLabels = np.random.permutation(np.array(range(maxNum))) % labels[i]
        print("Num of the label is : ", labels[i])
        lst=lst.append(tmpGroupBy.iloc[createdShuffleLabels], ignore_index=True)
        print("Done")
    # lst.to_csv('test1.csv', index=False)
    return lst

# 划分训练集和校验集
all_size = len(train_images)
# print(all_size)
train_size = int(all_size * 0.9)

train_images = shuffle(train_images)

train_image_list = train_images[:train_size]
val_image_list = train_images[train_size:]

# df = labelShuffling(train_image_list)

# df = shuffle(df)
df = train_image_list
train_image_path_list = df['image_id'].values
label_list = df['category_id'].values
label_list = paddle.to_tensor(label_list, dtype='int64')
train_label_list = paddle.nn.functional.one_hot(label_list, num_classes=137)

val_image_path_list = val_image_list['image_id'].values
val_label_list = val_image_list['category_id'].values
val_label_list = paddle.to_tensor(val_label_list, dtype='int64')
val_label_list = paddle.nn.functional.one_hot(val_label_list, num_classes=137)

# 定义数据预处理
data_transforms = T.Compose([
    T.Resize(size=(224, 224)),
    T.RandomHorizontalFlip(224),
    T.RandomVerticalFlip(224),
    T.Transpose(),    # HWC -> CHW
    T.Normalize(
        mean=[0, 0, 0],        # 归一化
        std=[255, 255, 255],
        to_rgb=True)    
])

In [ ]:
# 构建Dataset
class MyDataset(paddle.io.Dataset):
    """
    步骤一：继承paddle.io.Dataset类
    """
    def __init__(self, train_img_list, val_img_list,train_label_list,val_label_list, mode='train'):
        """
        步骤二：实现构造函数，定义数据读取方式，划分训练和测试数据集
        """
        super(MyDataset, self).__init__()
        self.img = []
        self.label = []
        # 借助pandas读csv的库
        self.train_images = train_img_list
        self.test_images = val_img_list
        self.train_label = train_label_list
        self.test_label = val_label_list
        if mode == 'train':
            # 读train_images的数据
            for img,la in zip(self.train_images, self.train_label):
                self.img.append('data/data98270/train/'+img)
                self.label.append(la)
        else:
            # 读test_images的数据
            for img,la in zip(self.train_images, self.train_label):
                self.img.append('data/data98270/train/'+img)
                self.label.append(la)

    def load_img(self, image_path):
        # 实际使用时使用Pillow相关库进行图片读取即可，这里我们对数据先做个模拟
        image = Image.open(image_path).convert('RGB')
        return image

    def __getitem__(self, index):
        """
        步骤三：实现__getitem__方法，定义指定index时如何获取数据，并返回单条数据（训练数据，对应的标签）
        """
        image = self.load_img(self.img[index])
        label = self.label[index]
        # label = paddle.to_tensor(label)
        
        return data_transforms(image), paddle.nn.functional.label_smooth(label)

    def __len__(self):
        """
        步骤四：实现__len__方法，返回数据集总数目
        """
        return len(self.img)

In [ ]:
#train_loader
train_dataset = MyDataset(train_img_list=train_image_path_list, val_img_list=val_image_path_list, train_label_list=train_label_list, val_label_list=val_label_list, mode='train')
train_loader = paddle.io.DataLoader(train_dataset, places=paddle.CPUPlace(), batch_size=128, shuffle=True, num_workers=0)

#val_loader

val_dataset = MyDataset(train_img_list=train_image_path_list, val_img_list=val_image_path_list, train_label_list=train_label_list, val_label_list=val_label_list, mode='test')
val_loader = paddle.io.DataLoader(val_dataset, places=paddle.CPUPlace(), batch_size=128, shuffle=True, num_workers=0)

### 模型训练

In [ ]:
# 调用飞桨框架的VisualDL模块，保存信息到目录中。
callback = paddle.callbacks.VisualDL(log_dir='visualdl_log_dir')

from visualdl import LogReader, LogWriter

args={
    'logdir':'./vdl',
    'file_name':'vdlrecords.model.log',
    'iters':0,
}

# 配置visualdl
write = LogWriter(logdir=args['logdir'], file_name=args['file_name'])
#iters 初始化为0
iters = args['iters'] 

#自定义Callback
class Callbk(paddle.callbacks.Callback):
    def __init__(self, write, iters=0):
        self.write = write
        self.iters = iters

    def on_train_batch_end(self, step, logs):

        self.iters += 1

        #记录loss
        self.write.add_scalar(tag="loss",step=self.iters,value=logs['loss'][0])
        #记录 accuracy
        self.write.add_scalar(tag="acc",step=self.iters,value=logs['acc'])

In [2]:
from work.res2net import Res2Net50_vd_26w_4s

# 模型封装
model_res = Res2Net50_vd_26w_4s(class_dim=137)

model = paddle.Model(model_res)

paddle.summary(mnist, (64, 3, 224, 224))

# 定义优化器

optim = paddle.optimizer.Adam(learning_rate=3e-4, parameters=model.parameters())

# 配置模型
model.prepare(
    optim,
    paddle.nn.CrossEntropyLoss(soft_label=True),
    Accuracy()
    )

model.load('work/Res2Net50_vd_26w_4s_pretrained.pdparams',skip_mismatch=True)

# 模型训练与评估
model.fit(train_loader,
        val_loader,
        log_freq=1,
        epochs=10,
        callbacks=Callbk(write=write, iters=iters),
        verbose=1,
        )

In [ ]:
# 保存模型参数
# model.save('Hapi_MyCNN_resume')  # save for training
model.save('Hapi_MyCNN1', False)  # save for inference

### 模型预测

In [ ]:
import os, time
import matplotlib.pyplot as plt
import paddle
from PIL import Image
import numpy as np
from paddle.vision import transforms

def load_image(img_path):
    '''
    预测图片预处理
    '''
    img = Image.open(img_path).convert('RGB')
    
    #resize
    img = img.resize((224, 224), Image.BILINEAR) #Image.BILINEAR双线性插值
    img = np.array(img).astype('float32')

    # HWC to CHW 
    img = img.transpose((2, 0, 1))
    
    #Normalize
    img = img / 255         #像素值归一化
    # print(img)
    # mean = [0.31169346, 0.25506335, 0.12432463]   
    # std = [0.34042713, 0.29819837, 0.1375536]
    # img[0] = (img[0] - mean[0]) / std[0]
    # img[1] = (img[1] - mean[1]) / std[1]
    # img[2] = (img[2] - mean[2]) / std[2]
    
    return img
use_gpu = False
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')
model = paddle.jit.load("Hapi_MyCNN1")
def read_img(path):

    img = Image.open(path)
    # img = img.convert("RGB")
    transform_valid = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0, 0, 0],  # 归一化
            std=[255, 255, 255],
            to_rgb=True)
    ])
    img_tensor = transform_valid(img).unsqueeze(0)

    return img_tensor

def infer_img(path, model):
    '''
    模型预测
    '''

    model.eval() 

  
    data = read_img(path)
    
    out = model(data)
    # print(out[0])
    # print(paddle.nn.functional.softmax(out)[0]) # 若模型中已经包含softmax则不用此行代码。

    lab = np.argmax(out.numpy())  #argmax():返回最大数的索引
    # label_pre.append(lab)
       
    return lab

In [ ]:
img_list = os.listdir('data/data98441/test/')
img_list.sort(key=lambda x: int(x[1:-4]))
# img_list

In [ ]:
pre_list = []

for i in range(len(img_list)):
    pre_list.append(infer_img(path='data/data98441/test/' + img_list[i], model=model))
    # print(i)
    # time.sleep(0.5) #防止输出错乱

In [ ]:
import pandas as pd

img = pd.DataFrame(img_list)
img = img.rename(columns = {0:"image_id"})
img['category_id'] = pre_list

img.to_csv('submit123.csv', index=False)

## 另一个解题思路


通过上面的柱状图，我们可以发现，这个数据集就是长尾分布。我们非常明显的知道，使用常规的图像分类网络应该是不可以的，所以我开了一个脑洞，能不能用两个网络，其中一个负责数量较多的那几个类别，另一个负责数量较少的那几个类别，最后再用一个全连接层把两个合在一起，这不就行了嘛！！！！于是我翻出了我去年的项目，[基于Densenet&Xception融合的102种鲜花识别](https://aistudio.baidu.com/aistudio/projectdetail/409180)，正当我沾沾自喜的时候，我发现有人已经研究了这个问题，并且还发了论文。。。所以我的顶会梦就这样碎了。

![](https://ai-studio-static-online.cdn.bcebos.com/1cb02138142847efabfc0be53b49355f670199ed29914b2dac2c33909da81854)



&emsp;&emsp;针对长尾分布中的不平衡分类问题，本文首次发现这些重新平衡方法能够实现令人满意的识别精度，这是因为它们可以显着地促进深度网络的分类学习。但是同时它们也在一定程度上破坏了学习到的深度特征的代表能力。 因此，本文提出了一个统一的双边分支网络(BBN)来同时处理表示学习和分类器学习，其中每个分支分别各自执行自己的职责 特别是，我们的BBN模型进一步配备了一种新的累积学习策略，其目的是首先学习通用模式，然后逐渐关注尾部数据。
       
       
       
&emsp;&emsp;工作初步首先做了验证实验，将深度网络的训练过程分为两步，分别执行特征学习和分类器学习。在第一个特征学习阶段，分别采用传统交叉熵训练方法，重加权和重采样去获得对应的特征表示。然后再第二个分类阶段，我们首先固定特征学习阶段的参数，然后训练这些网络的分类器（也就是全连接层），也是用第一阶段的三种方法。结果显示，当固定特征表示方式时，重平衡方法达到较低的错误率，表明他们能提升分类器学习。另一方面，当固定分类器学习方式， 对原始不平衡数据进行传统方法训练，可以根据其更好的特征带来更好的效果。 此外，重新平衡方法的更糟糕的结果证明了它们将损害特征学习。


## 总结


针对该类型比赛,大家在选定了一个baseline之后可以尝试各种技巧，包括学习率调整策略，模型调参等等。关于图像分类竞赛的一些技巧，大家可以查看我之前另一个项目，[图像分类竞赛技巧实战]。(https://aistudio.baidu.com/aistudio/projectdetail/1551646)  关于BBN这个网络，目前还没有基于飞桨实现的版本，反正我是没有发现。。。大家可以去github上拉取一下官方代码


**建议**

* 小白入坑，可独立完成一个比赛，不追求名次，但要渴望追求学习新知识。比赛开始优先使用最简单的模型（如ResNet18），快速跑完整个训练和预测流程。
* 要有一定毅力，不怕失败，比赛过程往往会踩到不少坑。数据扩增方法一定要反复尝试，会很大程度上影响模型精度。
* 有充足的时间，看相关论文，找灵感，有些domain的知识是必须有个基本概念认识。